# LIBRARIES IMPORT

In [ ]:
import CNN_lib
import CNN_utilities
import fig_lib 

import numpy as np
import matplotlib.pyplot as plt

import pickle
import json
import gc

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report


# SETTINGS

In [ ]:
d_type='processed'                                         # 'processed', 'synthetic'
video_d=True                                               # Video = True , Image = False 

data_split='rnd_patients'                                      # 'rnd_recs' 'rnd_patients'
data_augmentation= True

checking_on=True
t_btw_frm=0.05

In [ ]:
print('LIBRARY MODELS:')
print(' ')

for model_name in CNN_lib.MODELS.values(): print(model_name)

# -------- MODEL --------

model_from_CNN_lib,mdl= False,'video_conv3D'

opt='adam'
lss='binary_crossentropy'

# -------- TRAINNING --------

epochs=9

### PATHS

In [ ]:
checkpoint_path = "C:/PROJECTS\emboendo\CNN\python-machine-learning\emboendo_ML\embo_CNN_lab\_static/model_checkpoint.h5"

if d_type=='processed' : pkl_train_p='C:\PROJECTS\emboendo\CNN/pikles/processed_train_d.pkl'  
elif d_type=='synthetic' : pkl_train_p='C:\PROJECTS\emboendo\CNN/pikles/synthetic_train_d.pkl'   

if data_split == 'rnd_patients': json_fix_sq_p='C:\PROJECTS\emboendo\dicom_viewer\_static\Jsons\points_fix_d.json'

# GENERATE TRAIN DATA

In [ ]:
if data_split == 'rnd_patients':

    with open(json_fix_sq_p, 'r') as json_file:
        json_d = json.load(json_file)

    patients_d_df =  CNN_utilities.gen_patients_d_df(json_d)

In [ ]:
with open(pkl_train_p, 'rb') as pikle_file:

    save_trains= pickle.load(pikle_file)

x_train,y_train,rcs =  save_trains[0],save_trains[1],save_trains[2]

max_frm_n, HEIGHT, WIDTH=np.shape(x_train)[1],np.shape(x_train)[2],np.shape(x_train)[3]

### CHECK 1

In [ ]:
if checking_on:

    for x,y,r in zip(x_train,y_train,rcs):print(r,int(y),np.shape(x),type(x))

In [ ]:
if checking_on:

    rec_elm='I632'
    idx=rcs.index(rec_elm)
    print(y_train[idx])

In [ ]:
if checking_on:fig_lib.vid_show(x_train[idx],t_btw_frm=t_btw_frm)

### DATA SHUFFLE 1

In [ ]:
X_d,Y_d,recs=CNN_lib.shuffle(x_train,y_train,rcs)

# DATA SPLIT

In [ ]:
if data_split == 'rnd_recs': test_size=0.2            # 'rnd_recs' 'rnd_patients'

elif data_split == 'rnd_patients': val_pat_0,val_pat_1 = 9 , 5

In [ ]:
if data_split=='rnd_recs': 
    
    X_train_spl, X_eval_spl, Y_train_spl, Y_eval_spl ,recs_train,recs_eval=CNN_utilities.random_split_by_recs(X_d, Y_d,recs, test_size=test_size)

elif data_split == 'rnd_patients': 
    
    X_train_spl, X_eval_spl, Y_train_spl, Y_eval_spl ,recs_train,recs_eval=CNN_utilities.random_split_by_patients(patients_d_df,recs,X_d,Y_d, val_pat_0=val_pat_0, val_pat_1=val_pat_1)

print('Train data:',len(Y_train_spl),'Evaluation data:',len(Y_eval_spl))

In [ ]:
zeros_count = np.sum(Y_train_spl == 0)
ones_count = np.sum(Y_train_spl == 1)
zeros_count_val = np.sum(Y_eval_spl == 0)
ones_count_val = np.sum(Y_eval_spl == 1)

print(' Train data shape:', np.shape(X_train_spl),' Validation data shape:', np.shape(X_eval_spl))

CNN_lib.data_bars_plot(zeros_count,ones_count,zeros_count_val,ones_count_val)

### CHECK 2

In [ ]:
if checking_on:

    print('----------TRAINS----------')
    for x,y,r in zip(X_train_spl,Y_train_spl,recs_train):
        print(r,int(y),np.shape(x),type(x))

    print('----------VALIDATION----------')
    for x,y,r in zip(X_eval_spl,Y_eval_spl,recs_eval):
        print(r,int(y),np.shape(x),type(x))

In [ ]:
if checking_on:

    rec_elm='I632'

    try:

        idx=recs_train.index(rec_elm)
        fig_lib.vid_show(X_train_spl[idx],t_btw_frm=t_btw_frm)
        
    
    except:

        idx=recs_eval.index(rec_elm)
        fig_lib.vid_show(X_eval_spl[idx],t_btw_frm=t_btw_frm)
        


# DATA AUGMENTATION

In [ ]:
if data_augmentation:
    
    n_flip_1=0.7
    n_flip_0=0.07

    n_rot_1=0.7
    n_rot_0=0.07

    n_contr_1=0.4
    n_contr_0=0.04

    n_bright_1=0.4
    n_bright_0=0.04

    nf1,nf0 = round(n_flip_1*ones_count),round(n_flip_0*zeros_count)
    nr1,nr0 = round(n_rot_1*ones_count),round(n_rot_0*zeros_count)
    nc1,nc0 = round(n_contr_1*ones_count),round(n_contr_0*zeros_count)
    nb1,nb0 = round(n_bright_1*ones_count),round(n_bright_0*zeros_count)

    print(f'Flip 1: {nf1}, 0: {nf0}')
    print(f'Rotation 1: {nr1}, 0: {nr0}')
    print(f'Contrast 1: {nc1}, 0: {nc0}')
    print(f'brightness 1: {nb1}, 0: {nb0}')

In [ ]:

if data_augmentation:

    Flip_X_1,Flip_Y_1,Flip_recs_1=CNN_lib.main_aug_f(nf1,X_train_spl,Y_train_spl,recs,label=1,typ='Flip')
    Flip_X_0,Flip_Y_0,Flip_recs_0=CNN_lib.main_aug_f(nf0,X_train_spl,Y_train_spl,recs,label=0,typ='Flip')

    Rot_X_1,Rot_Y_1,Rot_recs_1=CNN_lib.main_aug_f(nr1,X_train_spl,Y_train_spl,recs,label=1,typ='Rotation')
    Rot_X_0,Rot_Y_0,Rot_recs_0=CNN_lib.main_aug_f(nr0,X_train_spl,Y_train_spl,recs,label=0,typ='Rotation')

    Cntr_X_1,Cntr_Y_1,Cntr_recs_1=CNN_lib.main_aug_f(nc1,X_train_spl,Y_train_spl,recs,label=1,typ='Contrast')
    Cntr_X_0,Cntr_Y_0,Cntr_recs_0=CNN_lib.main_aug_f(nc0,X_train_spl,Y_train_spl,recs,label=0,typ='Contrast')

    Bgr_X_1,Bgr_Y_1,Bgr_recs_1=CNN_lib.main_aug_f(nb1,X_train_spl,Y_train_spl,recs,label=1,typ='Brightness')
    Bgr_X_0,Bgr_Y_0,Bgr_recs_0=CNN_lib.main_aug_f(nb0,X_train_spl,Y_train_spl,recs,label=0,typ='Brightness')


In [ ]:
if data_augmentation:

    X_train_spl = np.concatenate((X_train_spl, Flip_X_1,Flip_X_0,Rot_X_1,Rot_X_0,Cntr_X_1,Cntr_X_0,Bgr_X_1,Bgr_X_0), axis=0)
    Y_train_spl = np.concatenate((Y_train_spl, Flip_Y_1,Flip_Y_0,Rot_Y_1,Rot_Y_0,Cntr_Y_1,Cntr_Y_0,Bgr_Y_1,Bgr_Y_0))
    
    recs_train = recs_train+Flip_recs_1+Flip_recs_0+Rot_recs_1+Rot_recs_0+Cntr_recs_1+Cntr_recs_0+Bgr_recs_1+Bgr_recs_0

    print(X_train_spl.shape,Y_train_spl.shape)
    print(X_eval_spl.shape,Y_eval_spl.shape)


### DATA SHUFFLE 2

In [ ]:
if data_augmentation: X_train,Y_train,recs_train_f=CNN_lib.shuffle(X_train_spl,Y_train_spl,recs_train)
else: X_train,Y_train,recs_train_f = X_train_spl,Y_train_spl,recs_train

X_eval=X_eval_spl
Y_eval=Y_eval_spl

In [ ]:
if data_augmentation:

    zeros_count = np.sum(Y_train == 0)
    ones_count = np.sum(Y_train == 1)
    zeros_count_val = np.sum(Y_eval == 0)
    ones_count_val = np.sum(Y_eval == 1)

    CNN_lib.data_bars_plot(zeros_count,ones_count,zeros_count_val,ones_count_val)

### CHECK 3

In [ ]:
if checking_on:

    for x,y,r in zip(X_train,Y_train,recs_train_f):print(r,int(y),np.shape(x),type(x))

In [ ]:
if checking_on:

    rec_elm='I632'

    try:

        idx=recs_train_f.index(rec_elm)
        fig_lib.vid_show(X_train[idx],t_btw_frm=t_btw_frm)
        
    except:print(f'{rec_elm} not in train data')

#  CNN TRAINNING

In [ ]:
gc.collect()

In [ ]:
if model_from_CNN_lib:
  
  if video_d: model = CNN_lib.lib_models(mdl,im_input_shp=(max_frm_n, HEIGHT, WIDTH, 1))
  else: model = CNN_lib.lib_models(mdl,im_input_shp=(HEIGHT, WIDTH,1))

else:

  input_shape=(max_frm_n, HEIGHT, WIDTH, 1)

  model = models.Sequential([
          layers.Conv3D(filters=16, kernel_size=(3, 3, 1), activation='relu',input_shape=input_shape),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=32, kernel_size=(3, 3, 1),  activation='relu'),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=32, kernel_size=(3, 3, 1),  activation='relu'),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=16, kernel_size=(1, 1, 5),  activation='relu'),
          layers.Flatten(),
          layers.Dropout(0.2),
          layers.Dense(64, activation='relu'),
          layers.Dense(1,activation='sigmoid')
          ])

model.summary()

In [ ]:
model.compile(optimizer=opt, loss=lss, metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss',   mode='min', verbose=1)

### TRAINNING

In [ ]:

history =model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_eval,Y_eval),callbacks=[checkpoint])

### EVALUATE MODEL

In [ ]:
CNN_lib.plot_train_eval(history,epochs)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_eval,Y_eval,verbose=2)

print(f'Loss: {test_loss}',f'Accuracy: {test_accuracy}')

In [ ]:
model = load_model(checkpoint_path)
test_loss, test_accuracy = model.evaluate(X_eval,Y_eval,verbose=2)
print(f'Loss: {test_loss}',f'Accuracy: {test_accuracy}')

In [ ]:
performance_d={'NAME':mdl, 'ACCURACY':test_accuracy, 'LOSS':test_loss, 'HEIGHT':HEIGHT,'WIDTH':WIDTH}

performance_d

# VALIDATION DATA

In [ ]:
predictions = model.predict(X_eval)
#print(predictions)


## ROC CURVE

In [ ]:
fpr_val, tpr_val, thresholds_val = roc_curve(Y_eval, predictions)
roc_auc_false = auc(fpr_val, tpr_val)

CNN_lib.plot_roc_curve(fpr_val,tpr_val,roc_auc_false)

#  FORECAST EVALUATION

In [ ]:
predictions_raw=[]

trsh=0.35

for pred in predictions:

    if pred < trsh:predictions_raw.append(0)
    else:predictions_raw.append(1)

predictions_raw=np.array(predictions_raw)


In [ ]:
conf_matrix = confusion_matrix(Y_eval, predictions_raw)

print("Matriz de Confusión:")
print(conf_matrix)

# Obtener el informe de clasificación
classification_report_str = classification_report(Y_eval, predictions_raw)
print("Informe de Clasificación:")
print(classification_report_str)



In [ ]:
true_classes = Y_eval  

correctly_classified_idx = np.where(predictions_raw == true_classes)[0]
incorrectly_classified_idx = np.where(predictions_raw != true_classes)[0]

correctly_classified_video_ids = [video_id for video_id in correctly_classified_idx]
incorrectly_classified_video_ids = [video_id for video_id in incorrectly_classified_idx]

num_zeros = np.count_nonzero(Y_eval == 0)
num_ones = np.count_nonzero(Y_eval == 1)


In [ ]:
correct_labels_val = [Y_eval[i] for i in correctly_classified_idx]

print(correct_labels_val)

In [ ]:

print(f"OK, {len(correctly_classified_video_ids)}")
print(f"BAD, {len(incorrectly_classified_video_ids)}")
print(f'Ratio forecast: {len(correctly_classified_video_ids)/len(incorrectly_classified_video_ids)}')
print(f'Ratio labeling: {max([num_ones,num_zeros])/min([num_ones,num_zeros])}')


## SINGLE FORECAST

In [ ]:
print(recs_eval)

In [ ]:
rec_elm='I765'
idx=recs_eval.index(rec_elm)

print(f' Record: {rec_elm} , index: {idx}, real label: {Y_eval_spl[idx]}')

In [ ]:
if checking_on:fig_lib.vid_show(X_eval[idx],t_btw_frm=t_btw_frm)

In [ ]:
nuevo_video = np.expand_dims(X_eval[idx], axis=0)

# Realizar la predicción
prediccion = model.predict(nuevo_video)
prediccion